# readme demo

Demo script for GNU Octave.

Tested environment:
- GNU Octave 6.4.0
- octave-kernel 0.35.1
- io 2.6.4
- Ubuntu 22.04 LTS

Dependencies
- [io package](https://gnu-octave.github.io/packages/io/)
- [jupyter-notebook](https://gnu-octave.github.io/packages/jupyter-notebook/)(optional; only if you want to run this script in GNU Octave)


You will need to install the [octave kernel for jupyter](https://pypi.org/project/octave-kernel/0.14.2/):
```bash
pip install octave_kernel
```

---

In [ ]:
clear

## Setup directories

In [ ]:
scriptDir = pwd % set this to the directory where this script is located
cd(scriptDir)
cd data
dataDir = pwd
cd(scriptDir)
cd keyMap
keyMapDir = pwd
cd(scriptDir)
cd output
outDir = pwd
cd ../..
homeDir = pwd

addpath(genpath(homeDir))

## Load data

Here the mat file `data.mat` is the export from Titta.

In [ ]:
cd(dataDir);
matFname = "data.mat";
tittaMat = load("-v7", matFname);

## Load key map

`keyMap` has to be in the format: `keyMap = {key1: value1, key2: value2, ...}`(in python-like notation).

The key is hardcoded in the script and must match. The value is user-defined.

Here, we will import from a csv file.

We assume that there are no different values for the same key.

In [ ]:
cd(dataDir)
keyMapFile = fopen('keymap_snake_case.csv');
keyMapTxt = textscan(keyMapFile, '%s%s', 'Delimiter', ',');
fclose(keyMapFile);
keyMap = containers.Map(keyMapTxt{1}, keyMapTxt{2});

## Create instance and run

Call the `Titta2delim` constructor with the following arguments:
- `tittaMat`: the mat file exported from Titta
- `keyMap`: the key map

You can add or overwrite `keyMap` after the instance is created using `addKeymap(keyMap)` method.

Call `main()` method to run the script.

In [ ]:
data = titta2delim(tittaMat, keyMap);
data.main()

### Customize for your own needs

You can customize the script for your own needs by inheritance/polymorphism.

For this demo, we have a `customExport` class that inherits from `Titta2delim`.

See `./customExport.m` for details.

In [ ]:
customData = customExport(tittaMat, keyMap);
customData.main()

### Export to csv

In Octave, Table is not implemented so all the properties that are table in Matlab are struct in Octave.

Here, we will create a simple function to export struct to csv file.

This function is slow so optimize it depending on your need.

In [ ]:
pkg load io

function struct2csv(inputStruct, fileName)
    outputCell = {};
    header = fieldnames(inputStruct);
    for i = 1:numel(header)
        outputCell = [outputCell inputStruct.(header{i})];
    end
    outputCell = [header'; outputCell];
    cell2csv(fileName, outputCell);
end

`TobiiLog` and `notifications` are not parsed in the script, therefore we export them differently.

In [ ]:
cd(outDir)
struct2csv(data.sessionInfo, 'sessionInfo.csv');
struct2csv(data.timeSeries, 'timeSeries.csv');
struct2csv(data.messages, 'messages.csv');
cell2csv('TobiiLog.csv', [fieldnames(data.TobiiLog) struct2cell(data.TobiiLog(:))]');
cell2csv('notifications.csv', [fieldnames(data.notifications) struct2cell(data.notifications(:))]');

### clean up

In [ ]:
rmpath(genpath(homeDir))